In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from IPython.display import clear_output

import PIL

import torch
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


from helpers import*
from lenshelpers import*

import warnings
warnings.filterwarnings('ignore') 

In [2]:
metadata = pd.DataFrame()
for i in np.arange(1,4):
    str_ID = "E"+str(i)+"P"+str(percent)"R"+str(ratio)
    [img, meta] = read_hdf5(str_ID, path = "data/dataSet/")
    metadata = pd.concat([metadata,meta], ignore_index=True)
    if i == 1:
        final_array = img
    else:
         final_array = np.concatenate((final_array, img))
metadata ['ID'] = np.arange(0,final_array.shape[0])
data_set = CombineDataset(metadata,'ID','class',final_array)

print('Reading Data Finished')
data_train, data_test = train_test_split(data_set,train_size=0.9,random_state=42)

Reading Data Finished


In [3]:
batch_size_train = 50
batch_size_test = 10
max_epoch = 10

loader_train = DataLoader(data_train, batch_size = batch_size_train, shuffle = True, 
                          num_workers = 0, drop_last=True)

loader_test = DataLoader(data_test, batch_size = batch_size_test, shuffle = True, 
                         num_workers = 0, drop_last=True)

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def compute_conv_output(in_size, padding, stride, kernel):
    layer_number = 0; k = 0
    out_size = (in_size + 2* padding[k] -kernel[k])/stride[k] + 1
    k = 1
    while True:
        layer_number += 1
        in_size = out_size
        out_size = (in_size + 2* padding[k] -kernel[k])/stride[k] + 1
        if k == 1: k = 0
        else: k = 1
        if out_size < 5:
            break
        
    return in_size, layer_number

class CustomCNN(nn.Module):
    
    def __init__(self, out_linear, out_conv, stride, padding, kernel, activation, batch_norm = False, in_size:int = 64, in_channel:int = 1, num_classes:int = 3):
        super(CustomCNN, self).__init__()
        self.out_size, self.conv_number =  compute_conv_output(in_size, padding, stride, kernel)
        self.nb_linear = out_linear.shape[0]
        self.out_linear = out_linear; self.out_linear[0] = out_conv[-1]*self.out_size*self.out_size
        
        self.out_linear[-1] = num_classes; self.batch_norm = batch_norm
        self.out_conv = out_conv; self.out_conv[0] = in_channel
        self.stride = stride; self.padding = padding
        self.kernel = kernel; 
        
        self.activation_layer = activation_func(activation)
        self.maxpool_layer = nn.MaxPool2d(kernel_size=kernel[1], stride=stride[1], padding=padding[1])
        
        self.layers_conv = nn.ModuleList()
        for i in np.arange(0, self.out_conv.shape[0]-1):
            self.layers_conv.append(nn.Conv2d(self.out_conv[i], self.out_conv[i+1], kernel_size=self.kernel[0], stride=self.stride[0], padding=self.padding[0]))
            #if batch_norm:
                #self.layers_conv.append(nn.BatchNorm2d(self.out_conv[i+1]))
        
        self.layers_linear = nn.ModuleList()
        for i in np.arange(0, self.out_linear.shape[0]-1):
            self.layers_conv.append(nn.Linear(self.out_linear[i], self.out_linear[i+1]))
            self.layers_conv.append(activation_func(self.activation))
            
        
        self.sigmoid = nn.Sigmoid()
    
    
    def activation_func(activation):
        return  nn.ModuleDict([
            ['relu', nn.ReLU(inplace=True)],
            ['leaky_relu', nn.LeakyReLU(negative_slope=0.01, inplace=True)],
            ['selu', nn.SELU(inplace=True)],
            ['none', nn.Identity()]])[activation] 
        
        
    def forward(self, x: torch.Tensor):
        for layer in self.layers_conv[:-1]:
            x = self.activation_layer(layer(x))
            x = self.maxpool_layer(x)
        x = torch.flatten(x, 1)
        for layer in self.layers_linear[:-1]:
            self.classifier = linear_layer(self.out_linear[i], self.out_linear[i+1])
            x = self.classifier(x)
        x = self.sigmoid(x)
        return x
        

In [5]:
strides_ = np.power(2,np.arange(0,4))
paddings_ = np.arange(0,4)
kernels_ = np.arange(3,10)
batch_norms_ = np.array([True, False])
activations_ = np.array(['relu','leaky_relu','selu','none'])
zero_paddings_ = np.array([0,5,10,20,30])
nb_linear_layers_ = np.arange(2,10)

range_Nunit_ = np.power(5,np.arange(1,6))


curr_stride = np.array([strides_[0],strides_[0]])
curr_padding = np.array([paddings_[0],paddings_[0]])
curr_kernel = np.array([kernels_[0],kernels_[0]])
curr_zero_padding = zero_paddings_[0]
curr_nb_linear_layer = nb_linear_layers_[0]
in_size = 64 + curr_zero_padding

out_size, layer_number = compute_conv_output(in_size, curr_padding, curr_stride, curr_kernel)

curr_conv = np.ones(layer_number)*range_Nunit_[3]
curr_linear = np.ones(curr_nb_linear_layer)*range_Nunit_[3]

curr_activation = activations_[0]
curr_batch = batch_norms_[0]
max_epoch = 10

In [6]:
net = CustomCNN (curr_linear, curr_conv, curr_stride, curr_padding, curr_kernel, curr_activation, batch_norm = curr_batch, in_size = in_size)
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

TypeError: new() received an invalid combination of arguments - got (numpy.float64, numpy.float64, numpy.int64, numpy.int64), but expected one of:
 * (*, torch.device device)
 * (torch.Storage storage)
 * (Tensor other)
 * (tuple of ints size, *, torch.device device)
 * (object data, *, torch.device device)


In [ ]:
for i in np.arange(0, 28):
    print(i)

In [ ]:
test_acc_metaxres = np.zeros(max_epoch)
for epoch in range(max_epoch):  # loop over the dataset multiple times

    train_net(loader_train, net, optimizer, criterion, epoch)
    mean_accuracy = test_net(loader_test,net)
    test_acc_metaxres[epoch] = mean_accuracy

    print("epoch: {:.3f}, accuracy: {:.3f} ".format(epoch, mean_accuracy))
print('Finished Training')